# Database Design Lab Use Cases Part 2: 

## 💨Setup💨

In [ ]:
# Import python packages

import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
USE ROLE DS5111_DBT;
USE DATABASE DS5111_SU24;
-- CREATE SCHEMA iy5sw;

USE SCHEMA iy5sw;


## 1) Which courses are currently included (active) in the program? Include the course mnemonic and course name for each.

In [ ]:

SELECT mnemonic, name
FROM COURSES_ERD
WHERE active = TRUE;

## 2) Which courses were included in the program, but are no longer active? Include the course mnemonic and course name for each.

In [ ]:

SELECT mnemonic, name
FROM COURSES_ERD
WHERE active = FALSE;

## 3) Which instructors are not current employees?

In [ ]:
SELECT name
FROM INSTRUCTOR_ERD
WHERE active = FALSE

## 4) For each course (active and inactive), how many learning outcomes are there?

In [ ]:
SELECT mnemonic, COUNT(learning_outcome) AS number_of_outcomes
FROM LEARNING_OUTCOMES_ERD
GROUP BY mnemonic

## 5) Are there any courses with no learning outcomes? If so, provide their mnemonics and names.

In [ ]:
SELECT COURSES_ERD.mnemonic, COURSES_ERD.name
FROM COURSES_ERD
LEFT JOIN LEARNING_OUTCOMES_ERD
ON COURSES_ERD.mnemonic = LEARNING_OUTCOMES_ERD.mnemonic
WHERE LEARNING_OUTCOMES_ERD.learning_outcome IS NULL

## 6) Which courses include SQL as a learning outcome? Include the learning outcome descriptions, course mnemonics, and course names in your solution.

In [ ]:
SELECT LEARNING_OUTCOMES_ERD.mnemonic, LEARNING_OUTCOMES_ERD.learning_outcome, COURSES_ERD.name
FROM COURSES_ERD
LEFT JOIN LEARNING_OUTCOMES_ERD
ON COURSES_ERD.mnemonic = LEARNING_OUTCOMES_ERD.mnemonic
WHERE LEARNING_OUTCOMES_ERD.learning_outcome LIKE '%SQL%'

## 7) Who taught course ds5100 in Summer 2021?

In [ ]:
SELECT INSTRUCTOR_ERD.name, COURSE_ASSIGNMENT_ERD.term, COURSE_ASSIGNMENT_ERD.mnemonic
FROM COURSE_ASSIGNMENT_ERD
LEFT JOIN INSTRUCTOR_ERD
ON COURSE_ASSIGNMENT_ERD.instructor_id = INSTRUCTOR_ERD.instructor_id
WHERE COURSE_ASSIGNMENT_ERD.mnemonic = 'ds5100' AND COURSE_ASSIGNMENT_ERD.term = 'SUMMER2021'

## 8) Which instructors taught in Fall 2021? Order their names alphabetically, making sure the names are unique.

In [ ]:
SELECT DISTINCT INSTRUCTOR_ERD.name, COURSE_ASSIGNMENT_ERD.term
FROM COURSE_ASSIGNMENT_ERD
LEFT JOIN INSTRUCTOR_ERD
ON COURSE_ASSIGNMENT_ERD.instructor_id = INSTRUCTOR_ERD.instructor_id
WHERE COURSE_ASSIGNMENT_ERD.term = 'FALL2021'
ORDER BY INSTRUCTOR_ERD.name

## 9) How many courses did each instructor teach in each term? Order your results by term and then instructor.

In [ ]:
SELECT COURSE_ASSIGNMENT_ERD.term, INSTRUCTOR_ERD.name, COUNT(COURSE_ASSIGNMENT_ERD.mnemonic) as number_of_courses_taught
FROM COURSE_ASSIGNMENT_ERD
LEFT JOIN INSTRUCTOR_ERD
ON COURSE_ASSIGNMENT_ERD.instructor_id = INSTRUCTOR_ERD.instructor_id
GROUP BY COURSE_ASSIGNMENT_ERD.term, INSTRUCTOR_ERD.name
ORDER BY COURSE_ASSIGNMENT_ERD.term, INSTRUCTOR_ERD.name

## 10a) Which courses had more than one instructor for the same term? Provide the mnemonic and term for each. Note this occurs in courses with multiple sections.

In [ ]:
SELECT COURSE_ASSIGNMENT_ERD.term, COURSE_ASSIGNMENT_ERD.mnemonic
FROM COURSE_ASSIGNMENT_ERD
GROUP BY COURSE_ASSIGNMENT_ERD.term, COURSE_ASSIGNMENT_ERD.mnemonic
HAVING COUNT(COURSE_ASSIGNMENT_ERD.instructor_id) > 1


## 10b) For courses with multiple sections, provide the term, course mnemonic, and instructor name for each. Hint: You can use your result from 10a in a subquery or WITH clause.

In [ ]:
WITH multiple_sections AS (
    SELECT COURSE_ASSIGNMENT_ERD.term, COURSE_ASSIGNMENT_ERD.mnemonic
    FROM COURSE_ASSIGNMENT_ERD
    GROUP BY COURSE_ASSIGNMENT_ERD.term, COURSE_ASSIGNMENT_ERD.mnemonic
    HAVING COUNT(COURSE_ASSIGNMENT_ERD.instructor_id) > 1
)

SELECT COURSE_ASSIGNMENT_ERD.term, COURSE_ASSIGNMENT_ERD.mnemonic, INSTRUCTOR_ERD.name
FROM COURSE_ASSIGNMENT_ERD
LEFT JOIN INSTRUCTOR_ERD
ON COURSE_ASSIGNMENT_ERD.instructor_id = INSTRUCTOR_ERD.instructor_id
JOIN multiple_sections 
ON COURSE_ASSIGNMENT_ERD.term = multiple_sections.term AND COURSE_ASSIGNMENT_ERD.mnemonic =  multiple_sections.mnemonic 
